In [2]:
import os
import pandas as pd
import janitor

pef = os.path.join('..', 'data', '2022.xlsx')

pef_df=pd.read_excel(pef)
pef_df= pef_df.clean_names()


### Gráfica por institución

In [15]:
import ipywidgets as widgets
import plotly.graph_objects as go

# Lista de desc_ramo
ramos = pef_df['desc_ramo'].unique().tolist()

# Widget Dropdown
ramo_dropdown = widgets.Dropdown(
    options=ramos,
    value=ramos[0],
    description='Ramo:',
)

# Función para generar gráfico
def plot_graph(ramo):
    ramo_df = pef_df[pef_df['desc_ramo'] == ramo]
    ramo_df = ramo_df.groupby('desc_ur').agg(
        monto_aprobado=('monto_aprobado', 'sum'),
        monto_pagado=('monto_pagado', 'sum')
    ).sort_values('monto_aprobado', ascending=True)

    # Creando las barras para el Monto Aprobado
    trace1 = go.Bar(
        y=ramo_df.index,
        x=ramo_df['monto_aprobado'] / 1_000_000,  # Convertir a millones
        name='Monto Aprobado',
        orientation='h',
        marker_color='blue',
        text=ramo_df['monto_aprobado'] / 1_000_000,  # Convertir a millones
        textposition='outside',
        texttemplate='%{text:,.0f} Millones'  # Modificar template para incluir 'Millones'
    )

    # Creando las barras para el Monto Pagado
    trace2 = go.Bar(
        y=ramo_df.index,
        x=ramo_df['monto_pagado'] / 1_000_000,  # Convertir a millones
        name='Monto Pagado',
        orientation='h',
        marker_color='green',
        text=ramo_df['monto_pagado'] / 1_000_000,  # Convertir a millones
        textposition='outside',
        texttemplate='%{text:,.0f} Millones'  # Modificar template para incluir 'Millones'
    )

    # Agregamos las barras en la gráfica
    data = [trace1, trace2]

    # Creamos el layout
    layout = go.Layout(
        title=f'{ramo} - Comparación entre el Monto Aprobado y el Monto Pagado (en millones) 2022',
        barmode='group',
        xaxis=dict(
            title='Monto (en millones)',
            tickformat=","
        ),
        yaxis=dict(title='Concepto', automargin=True),
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(
            size=14,
            color="black"
        ),
        annotations=[
            dict(
                x=0,
                y=-0.15,
                showarrow=False,
                text="Fuente: Elaboración propia con datos de Transparencia Presupuestaria",
                xref="paper",
                yref="paper",
                font=dict(size=12)
            )
        ]
    )

    # Creamos el objeto Figure y visualizamos
    fig = go.Figure(data=data, layout=layout)
    fig.update_layout(
        separators=".,",
        width=2000, 
        height=1200
    )
    fig.show()

# Función para manejar el evento de cambio en el Dropdown
def on_ramo_change(change):
    plot_graph(change.new)

# Asociamos la función 'on_ramo_change' al evento 'value' del Dropdown
ramo_dropdown.observe(on_ramo_change, names='value')

# Mostramos el Dropdown
display(ramo_dropdown)

# Generamos el primer gráfico
#plot_graph(ramos[0])


Dropdown(description='Ramo:', options=('Instituto Nacional Electoral', 'Comisión Nacional de los Derechos Huma…